# 項目分析

テストを実施したあと、モデルをあてはめてパラメータを推定する前の段階で簡単に計算できる指標群で項目の良し悪しを考察できる。

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def generate_irt_2pl_data(num_persons=1000, num_items=20, seed=42):
    """2PLM用の二値反応サンプルデータセットを生成する"""
    np.random.seed(seed)

    # 潜在能力 θ を標準正規分布から生成
    theta = np.random.normal(loc=0, scale=1, size=num_persons)

    # 識別力 a ~ lognormal(μ=0, σ=0.2)
    a = np.random.lognormal(mean=0, sigma=0.2, size=num_items)

    # 困難度 b ~ N(0, 1)
    b = np.random.normal(loc=0, scale=1, size=num_items)

    # 応答行列を初期化
    response_matrix = np.zeros((num_persons, num_items))

    for i in range(num_persons):
        for j in range(num_items):
            p = 1 / (1 + np.exp(-a[j] * (theta[i] - b[j])))  # 正答確率
            response_matrix[i, j] = np.random.binomial(n=1, p=p)  # 0 or 1

    # DataFrameとして返す（行=受験者, 列=項目）
    responses = pd.DataFrame(response_matrix, columns=[f"item_{j+1}" for j in range(num_items)]).astype(int)
    return responses, a, b, theta



In [7]:
responses, a, b, theta = generate_irt_2pl_data(num_persons=500, num_items=10)

## 項目困難度 / 通過率

古典テスト理論における項目困難度は、正答率のこと。

わかりやすさのために「通過率」と呼ばれることもある。各設問の正答の容易さを示す。


In [13]:
responses.mean()

item_1     0.430
item_2     0.516
item_3     0.538
item_4     0.720
item_5     0.634
item_6     0.364
item_7     0.382
item_8     0.674
item_9     0.466
item_10    0.396
dtype: float64

## 項目識別力

古典テスト理論における項目識別力は、素点（テスト得点）と各項目得点の相関をみるもの。

テストで測りたい能力を大まかに表すものである素点に対し、各項目が相関している（能力を測るのに貢献している）ことを確かめる。


In [15]:
df = responses.copy()
df["raw_score"] = responses.sum(axis=1)

from ordinalcorr import biserial

results = []
for item_col in df.filter(like="item_").columns:
    results.append({
        "item": item_col,
        "corr": biserial(df["raw_score"], df[item_col]),
    })
pd.DataFrame(results).round(3)

,item,corr
0,item_1,0.642
1,item_2,0.747
2,item_3,0.557
3,item_4,0.659
4,item_5,0.597
5,item_6,0.625
6,item_7,0.633
7,item_8,0.715
8,item_9,0.536
9,item_10,0.620


## 項目特性図 / トレースライン

各被験者の素点を横軸に、素点ごとの各選択肢の選択割合を縦軸にした折れ線グラフ。



In [ ]:
# 簡易的に2択の設問と仮定して、正解のほうを選べた率を表している
df = responses.copy()
df["raw_score"] = responses.sum(axis=1)
df["raw_score"] = pd.qcut(df["raw_score"], 5).astype(str)

item_cols = df.filter(like="item_").columns
stats = df.groupby("raw_score")[item_cols].mean().reset_index()

import seaborn as sns
sns.lineplot(
    data=stats.melt(id_vars="raw_score", value_name="pass_rate"),
    x="raw_score", y="pass_rate", hue="variable",
    marker="o"
)

<Axes: xlabel='raw_score', ylabel='pass_rate'>